# Notebook 4 - g-computation method (student notebook)

Let's reconsider the LaLonde dataset as we try to implement a g-computation implementation from scratch. 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from statsmodels.distributions.empirical_distribution import ECDF

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
# Windows users: replace the following line with `df = pd.read_csv(r"..\..\data\lalonde.csv")`
df = pd.read_csv("../../data/lalonde.csv")

To remind you, the LaLonde dataset should contain the following 12 columns:

age<br>
   $\;\;\;\;\;\;$age in years.<br>
educ<br>
    $\;\;\;\;\;\;$years of schooling.<br>
black<br>
    $\;\;\;\;\;\;$indicator variable for blacks.<br>
hisp<br>
    $\;\;\;\;\;\;$indicator variable for Hispanics.<br>
married<br>
    $\;\;\;\;\;\;$indicator variable for martial status.<br>
nodegr<br>
    $\;\;\;\;\;\;$indicator variable for high school diploma.<br>
re74<br>
    $\;\;\;\;\;\;$real earnings in 1974.<br>
re75<br>
    $\;\;\;\;\;\;$real earnings in 1975.<br>
re78<br>
    $\;\;\;\;\;\;$real earnings in 1978.<br>
u74<br>
    $\;\;\;\;\;\;$indicator variable for earnings in 1974 being zero.<br>
u75<br>
    $\;\;\;\;\;\;$indicator variable for earnings in 1975 being zero.<br>
treat<br>
    $\;\;\;\;\;\;$an indicator variable for treatment status.<br>

In [ ]:
df.head()

In [ ]:
df['ID'] = range(0,445)

In [ ]:
df = df[['ID', 'age', 'educ', 'black', 'hisp', 'married', 're78', 'treat']]

In [ ]:
# Let's scale our continuous covariates
features = df[['age', 'educ']]

# Use scaler of choice; here Standard scaler is used
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

df[['scaled_age', 'scaled_educ']] = features

## First, let's train a predictive model with scikit-learn. This model should take in the treatment variable and covariates, and predict the outcome variable (i.e. `re78`)

If you're curious about how well the model performs that's probably a good thing to be curious about. Let's split our small dataset into test and training sets. Now, admittedly, we have a tiny dataset, but we're doing this to show that standard ML evaluation methods are still relevant here.


In [ ]:
train_df, test_df = train_test_split(df, test_size = 0.20, random_state = 512)

In [ ]:
features = ['scaled_age', 'scaled_educ', 'black', 'hisp', 'married', 'treat']

Given the tiny data, let's use simple ML like a multiple linear regression model

In [ ]:
temp_model = LinearRegression().fit(X = train_df[features], y = train_df['re78'])

In [ ]:
mean_squared_error(temp_model.predict(test_df[features]).reshape(-1,1), test_df['re78'].values.reshape(-1,1))

You could compare different model types here to see which performs best in a hold-out set. Not surprisingly, given the small dataset, it's going to be hard to find a model that performs well. Ideally, we want a fairly good predicive model so that we can see accurate counterfactual outcomes. Anyways, this is just a demonstration so the model doesn't have to perform outstandingly well. Let's now train on the whole dataset.

In [ ]:
model = LinearRegression().fit(X = df[features], y = df['re78'])

## Now that we have a model we can start making predictions to see counterfactual outcomes

<div class="alert alert-success">
    <h3>EXERCISE: Take the full dataset and "force" everyone to have received the treatment, what is the mean value of real earnings from 1978 that you predict with your model?</h3>
</div>

In [ ]:
df_treat = df.copy()
df_treat['treat'] = ____

In [ ]:
mean_treat = model.predict(df_treat[features]).mean()

In [ ]:
print(f"Mean outcome value is {round(mean_treat, 2)}")

<div class="alert alert-success">
    <h3>EXERCISE: Now "force" everyone to have not received the treatment. What is the mean outcome value you get now?</h3>
</div>

In [ ]:
df_ctrl = df.copy()
df_ctrl['treat'] = ______

In [ ]:
mean_ctrl = model.predict(df_ctrl[features]).mean()

In [ ]:
print(f"Mean outcome value is {round(mean_ctrl, 2)}")

## The difference (6283.43 - 4601.56 = 1681.87) between these two represents the average treat effect (ATE). This is the expected mean difference in real earnings between a population that received the treatment and the same population that didn't receive the treatment. Confounders are adjusted for. 

## You could obtain confidence intervals around this estimate by taking bootstrap replicates of the original dataset, training a new model on each replicate, and then calculating the ATE for each replicate. The 2.5th and 97.5th percentiles of these values would represent the bounds of your 95% confidence interval.

<div class="alert alert-success">
    <h3>EXERCISE: Can you calculate the average treatment effect among the treated (ATT)? Hint: just as the metric's name suggests, you would focus exclusively on the originally treated group, and see how a receiving and not receiving the treatment among that group would play out.</h3>
</div>

## Interestingly the ATT and ATE are the same in this exercise. Don't expected that to be the case often. We used a tiny dataset, with a small number of features, with a very simple linear model. In real life having different ATE, ATT, and ATU values are the norm.